Below is a complete SARSA tutorial written in markdown. It starts with an explanation of the SARSA algorithm in pseudocode (verbal description) and then shows a detailed Python example using a simple maze problem. The SARSA update equation and all key components are explained.

---

## 1. SARSA Algorithm Overview

**SARSA** stands for State-Action-Reward-State-Action, and it’s an on-policy Temporal-Difference (TD) control method. The key difference from Q-learning is that SARSA updates its Q-values using the actual action taken in the next state (following the current policy), whereas Q-learning uses the maximum estimated value for the next state.

### SARSA Pseudocode (Verbal Description)

1. **Initialize** all Q(s, a) arbitrarily for all state-action pairs.
2. For each **episode** (from start to termination):
   - Set initial state, s.
   - Choose an initial action, a, using an *epsilon-greedy* policy based on Q.
   - While s is not terminal:
     - Execute action a, observe reward r and next state s′.
     - Choose the next action a′ in s′ following the same policy (epsilon-greedy).
     - **Update** Q(s, a) using the SARSA equation:  
       Q(s, a) ← Q(s, a) + α [r + γ · Q(s′, a′) − Q(s, a)]  
         - Here, α is the learning rate.
         - γ is the discount factor.
     - Set (s, a) ← (s′, a′).

3. Repeat for many episodes so that Q(s, a) approximates the optimal action-value function while following the current policy.

---

## 2. Detailed Python Tutorial Using a Simple Maze Problem

In this example, we use a maze environment where:
- The maze is a 2D grid defined as a NumPy array.
- 0 represents a free cell, and 1 represents a wall.
- The agent’s actions are: up, down, left, right.
- The agent receives a reward of –1 for every step and +100 when the goal is reached.
- If the agent attempts to move into a wall or outside the maze, it remains in the same cell.

Below is the Python source code implementing the SARSA algorithm.


In [43]:
#!/usr/bin/env python3
import numpy as np
import random

# ---------------------------
# Maze Environment Definition
# ---------------------------
class MazeEnv:
    def __init__(self):
        # Define the maze layout: 0 = free space, 1 = wall.
        # Example: a 6x6 maze.
        self.maze = np.array([
            [0, 0, 0, 1, 0, 0],
            [0, 1, 0, 1, 0, 1],
            [0, 1, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 0],
            [1, 0, 0, 0, 1, 0],
            [0, 0, 1, 0, 0, 0]
        ])
        self.n_rows, self.n_cols = self.maze.shape
        
        # Define start and goal positions.
        self.start_state = (0, 0)   # Top-left corner.
        self.goal_state = (5, 5)    # Bottom-right corner.
        
        # Define available actions: up, down, left, right.
        self.action_space = {
            0: (-1, 0),   # Up
            1: (1, 0),    # Down
            2: (0, -1),   # Left
            3: (0, 1)     # Right
        }
        self.n_actions = len(self.action_space)
        self.current_state = self.start_state

    def reset(self):
        """Reset the environment to the initial state."""
        self.current_state = self.start_state
        return self.current_state

    def step(self, action):
        """
        Take one step in the maze.
        Parameters: action (0: up, 1: down, 2: left, 3: right)
        Returns: next_state, reward, done (True if goal is reached)
        """
        # Get the movement delta for the action.
        delta = self.action_space[action]
        next_state = (self.current_state[0] + delta[0],
                      self.current_state[1] + delta[1])
        
        # Boundary check: if out of bounds, remain in current state.
        if (next_state[0] < 0 or next_state[0] >= self.n_rows or
            next_state[1] < 0 or next_state[1] >= self.n_cols):
            next_state = self.current_state
        # Check if hitting a wall
        elif self.maze[next_state] == 1:
            next_state = self.current_state
        
        # Define reward: +100 if goal reached, -1 for each step.
        if next_state == self.goal_state:
            reward = 100
            done = True
        else:
            reward = -1
            done = False
        
        self.current_state = next_state
        return next_state, reward, done

    def state_to_index(self, state):
        """Convert a (row, col) state into an index for Q table."""
        return state[0] * self.n_cols + state[1]

    def print_policy(self, Q):
        """Print the learned policy as arrows for each cell (except walls)."""
        directions = {0: '↑', 1: '↓', 2: '←', 3: '→'}
        policy = np.full(self.maze.shape, ' ')
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                if self.maze[r, c] == 1:
                    policy[r, c] = '█'
                elif (r, c) == self.goal_state:
                    policy[r, c] = 'G'
                else:
                    state_index = self.state_to_index((r, c))
                    best_action = np.argmax(Q[state_index])
                    policy[r, c] = directions[best_action]
        for row in policy:
            print(' '.join(row))


# ---------------------------
# SARSA Algorithm Parameters
# ---------------------------
alpha = 0.1         # Learning rate
gamma = 0.9         # Discount factor
epsilon = 0.2       # Exploration probability for epsilon-greedy
num_episodes = 500  # Number of episodes for training

# Initialize the environment and Q table.
env = MazeEnv()
n_states = env.n_rows * env.n_cols
n_actions = env.n_actions
Q = np.zeros((n_states, n_actions))  # Q table with dimensions: states x actions

# SARSA update equation:
# Q(s, a) ← Q(s, a) + α * [r + γ * Q(s′, a′) − Q(s, a)]

# ---------------------------
# SARSA Training Loop
# ---------------------------
for episode in range(num_episodes):
    state = env.reset()
    state_index = env.state_to_index(state)
    
    # Choose initial action using an epsilon-greedy policy.
    if random.uniform(0, 1) < epsilon:
        action = random.choice(list(env.action_space.keys()))
    else:
        action = np.argmax(Q[state_index])
    
    done = False
    
    while not done:
        # Take the action, observe reward and next state.
        next_state, reward, done = env.step(action)
        next_state_index = env.state_to_index(next_state)
        
        # Choose next action based on the current policy (epsilon-greedy).
        if not done:
            if random.uniform(0, 1) < epsilon:
                next_action = random.choice(list(env.action_space.keys()))
            else:
                next_action = np.argmax(Q[next_state_index])
        else:
            # If episode is finished, there is no next action.
            next_action = None
        
        # SARSA update.
        # If next_action is None (terminal state), Q(next_state, next_action) is 0.
        q_current = Q[state_index, action]
        if next_action is not None:
            q_target = reward + gamma * Q[next_state_index, next_action]
        else:
            q_target = reward  # For terminal state
            
        Q[state_index, action] = q_current + alpha * (q_target - q_current)
        
        # Move to the next state and action.
        state_index = next_state_index
        action = next_action if next_action is not None else action

    # Optionally print progress every 100 episodes.
    if (episode + 1) % 100 == 0:
        print("Episode {} completed.".format(episode + 1))

# ---------------------------
# Display the Results
# ---------------------------
print("\nLearned Q-table:")
print(Q)

print("\nLearned Policy (arrows indicate best action):")
env.print_policy(Q)

# ---------------------------
# Test the Learned Policy
# ---------------------------
print("\nTest Run:")
state = env.reset()
steps = 0
path = [state]
done = False

while not done and steps < 50:
    state_index = env.state_to_index(state)
    action = np.argmax(Q[state_index])
    state, reward, done = env.step(action)
    path.append(state)
    steps += 1

print("Path taken:", path)
if done:
    print("Goal reached in {} steps!".format(steps))
else:
    print("Failed to reach goal within step limit.")



Episode 100 completed.
Episode 200 completed.
Episode 300 completed.
Episode 400 completed.
Episode 500 completed.

Learned Q-table:
[[  8.53590385  15.81467581   8.38920066   3.78890365]
 [ -3.90946426  -3.88820822   9.80354611  -3.50214467]
 [ -3.45527943  -1.78592367  -3.46122563  -3.35284734]
 [  0.           0.           0.           0.        ]
 [ -0.97647478  -1.04523739  -1.02292406  -0.96721852]
 [ -0.95935281  -0.91677845  -1.00773746  -0.99235258]
 [  7.73899746  22.43052139  12.14505378  11.82572409]
 [  0.           0.           0.           0.        ]
 [ -2.85996684   1.01707719  -2.81547598  -2.78406192]
 [  0.           0.           0.           0.        ]
 [ -1.01270887  -0.54262808  -1.08980124  -1.10441679]
 [  0.           0.           0.           0.        ]
 [ 13.22927893  29.13458281  17.4641735   16.64728312]
 [  0.           0.           0.           0.        ]
 [ -2.28728335  -1.4827975   -2.00583841   7.20109631]
 [ -1.6902753   -0.76220184  -1.39156456  

## Explanation of the Code

### Environment (MazeEnv)
- **Maze Setup**: The maze is defined using a NumPy array where:
  - 0 represents an open cell.
  - 1 represents a wall.
- **Actions**: The agent can move in four directions (up, down, left, right). Moves that hit a wall or go out of bounds leave the agent in the same state.
- **Rewards**: The agent gets –1 per step and a reward of +100 on reaching the goal.
- **Helper Methods**:
  - `reset()`: Initializes the environment.
  - `step(action)`: Executes an action and returns the next state, reward, and a boolean indicating if the goal is reached.
  - `state_to_index(state)`: Converts the (row, col) state into a unique index (used to index the Q table).
  - `print_policy(Q)`: Displays the learned policy, marking the best action from the Q table with arrows.

### SARSA Training Loop
- **Initialization**: The Q table is initialized with zeros for all state-action pairs.
- **Epsilon-Greedy Policy**: Both the initial action and the next actions are chosen using an epsilon-greedy strategy, which allows exploration.
- **SARSA Update**:  
  The Q-value is updated with the following equation:  
  Q(s, a) ← Q(s, a) + α · [r + γ · Q(s′, a′) − Q(s, a)]  
  Here:
  - s, a: current state and action.
  - r: reward received after taking action a.
  - s′, a′: next state and the action chosen in that next state.
  - α: learning rate.
  - γ: discount factor.
- **Episode Termination**: If the goal is reached, the episode ends.

### Testing the Learned Policy
- After training, the code runs a test episode where the agent always picks the action with the highest Q-value to demonstrate the learned path from the start to the goal.

---
# Difference between Q-learning and SARSA
The key difference between Q-learning and SARSA is that Q-learning is an off-policy algorithm while SARSA is an on-policy algorithm.

- In **Q-learning**, the update rule uses the maximum estimated Q-value for the next state (i.e., it assumes the best possible action will be taken next). This means that even if the agent follows an exploratory (non-greedy) behavior, the update still uses a greedy estimate:  

  Q(s, a) ← Q(s, a) + α [r + γ · **maxₐ′ Q(s′, a′)** − Q(s, a)]

- In **SARSA**, the update rule uses the actual action that the agent takes in the next state according to its current (often exploratory) policy. This means that the learning is affected by the current policy’s propensity for exploration:  

  Q(s, a) ← Q(s, a) + α [r + γ · **Q(s′, a′)** − Q(s, a)]

In summary, Q-learning is focused on learning the optimal policy regardless of the behavior policy (off-policy), while SARSA learns the value of following the current policy (on-policy).